## MNIST CNN

In [18]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Weight initialization

To create this model, we're going to need to create a lot of weights and biases. One should generally initialize weights with a small amount of noise for symmetry breaking, and to prevent 0 gradients. Since we're using ReLU neurons, it is also good practice to initialize them with a slightly positive initial bias to avoid "dead neurons".

In [2]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial,name="Weights")

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial,name="biases")

### Convolution and pooling

How do we handle the boundaries? What is our stride size? In this example, we're always going to choose the vanilla version. Our convolutions uses a stride of one and are zero padded so that the output is the same size as the input. Our pooling is plain old max pooling over 2x2 blocks.

**tf.nn.conv2d**(
   
    input,
    filter,
    strides,  
    padding,
    use_cudnn_on_gpu=True,
    data_format='NHWC',
    dilations=[1, 1, 1, 1],
    name=None
)

**Args:**

> **input**: A Tensor. Must be one of the following types: half, bfloat16, float32. A 4-D tensor. The dimension order is interpreted according to the value of data_format, see below for details.

>**filter**: A Tensor. Must have the same type as input. A 4-D tensor of shape [filter_height, filter_width, in_channels, out_channels]

>**strides**: A list of ints. 1-D tensor of length 4. The stride of the sliding window for each dimension of input. The dimension order is determined by the value of data_format [num_batch, Height,width,channels].

>**padding**: A string from: "SAME"(zero padding), "VALID"(No padding. drop elements). The type of padding algorithm to use.

In [3]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

### Building the graph

We start building the computation graph by creating nodes for the input images and target output classes.

In [4]:
x = tf.placeholder(tf.float32,[None,784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

We first reshape x to a 4d tensor, with the second and third dimensions corresponding to image width and height, and the final dimension corresponding to the number of color channels.

In [5]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

**CNN layers**

We can now implement our first layer. It will consist of convolution, followed by max pooling. The convolution will compute 32 features for each 5x5 patch. Its weight tensor will have a shape of [5, 5, 1, 32]. The first two dimensions are the patch size, the next is the number of input channels, and the last is the number of output channels. We will also have a bias vector with a component for each output channel.

We then convolve x_image with the weight tensor, add the bias, apply the ReLU function, and finally max pool. The max_pool_2x2 method will reduce the image size to 14x14.

In [6]:
with tf.variable_scope("conv1"):
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    
with tf.variable_scope("maxpool1"):
    h_pool1 = max_pool_2x2(h_conv1)

print("conv1", h_conv1.shape)
print("pool1", h_pool1.shape)

conv1 (?, 28, 28, 32)
pool1 (?, 14, 14, 32)


In order to build a deep network, we stack several layers of this type. The second layer will have 64 features for each 5x5 patch.

In [7]:
with tf.name_scope("conv2"):
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
with tf.variable_scope("maxpool2"):
    h_pool2 = max_pool_2x2(h_conv2)

print("conv2", h_conv2.shape)
print("pool2", h_pool2.shape)

conv2 (?, 14, 14, 64)
pool2 (?, 7, 7, 64)


**Dense Layers**

Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons to allow processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, and apply a ReLU.

In [8]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

print("pool2_flat", h_pool2_flat.shape)

pool2_flat (?, 3136)


In [9]:
with tf.name_scope("fc"):
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

print("fc1", h_fc1.shape)

fc1 (?, 1024)


To reduce overfitting, we will apply dropout before the readout layer. We create a placeholder for the probability that a neuron's output is kept during dropout. This allows us to turn dropout on during training, and turn it off during testing

In [10]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

with tf.name_scope("output"):
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])
    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

print("y", y_conv.shape)

y (?, 10)


### Training

Our loss function is the cross-entropy between the target and the softmax activation function applied to the model's prediction.

In [11]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y_conv))
ce_summary = tf.summary.scalar("Cross_entropy_loss",cross_entropy)

For training, we will use ADAM optimizer

In [12]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

### Validation

In [13]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
ac_summary = tf.summary.scalar("accuracy",accuracy)

### Run the graph

Test accuracy is 99.2% for 20K iterations

In [17]:
#merged = tf.summary.merge_all()
merged_ac = tf.summary.merge([ac_summary,])
merged_ce = tf.summary.merge([ce_summary,])

with tf.Session() as sess:

  writer = tf.summary.FileWriter("log/MNIST_CNN/",sess.graph)

  sess.run(tf.global_variables_initializer())
  for i in range(3000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
      summary_ac, train_accuracy = sess.run([merged_ac,accuracy], feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
      writer.add_summary(summary_ac,i)
    summary_ce, _ = sess.run([merged_ce,train_step],feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    writer.add_summary(summary_ce,i)
   

  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

  
  

step 0, training accuracy 0.06
step 100, training accuracy 0.88
step 200, training accuracy 0.94
step 300, training accuracy 0.98
step 400, training accuracy 0.9
step 500, training accuracy 0.98
step 600, training accuracy 0.96
step 700, training accuracy 1
step 800, training accuracy 0.96
step 900, training accuracy 0.92
step 1000, training accuracy 0.98
step 1100, training accuracy 0.98
step 1200, training accuracy 0.98
step 1300, training accuracy 0.94
step 1400, training accuracy 0.94
step 1500, training accuracy 1
step 1600, training accuracy 1
step 1700, training accuracy 0.94
step 1800, training accuracy 1
step 1900, training accuracy 0.98
step 2000, training accuracy 0.94
step 2100, training accuracy 1
step 2200, training accuracy 0.96
step 2300, training accuracy 0.94
step 2400, training accuracy 0.96
step 2500, training accuracy 0.96
step 2600, training accuracy 0.96
step 2700, training accuracy 0.96
step 2800, training accuracy 1
step 2900, training accuracy 1
test accuracy 

### Graph

![graph](graph_CNN.png)

![acc](acc.png)
![ce](ce_loss.png)